In [1]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)
from emo_utils import *
import glob
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
stop_words = set(stopwords.words('english'))
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
lemma = nltk.wordnet.WordNetLemmatizer()

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Aditi\Anaconda3\envs\keras\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
f=open('data/ag_news_csv/train.csv','r')
lines=f.readlines()
Y=[]
X=[]
for line in lines:
    line=line.split(',')
#     -1 to start index from 0
    Y.append(int(line[0].strip("\""))-1)
    text=line[1].strip("\"")+" "+line[2].strip("\"")
    text=re.sub('[^ a-zA-Z]', '', text)
    text= pattern.sub('', text)
    result=""
    for word in text.split():
        if len(word)>=4:
            result+=lemma.lemmatize(word)+" "
    text=result
    X.append(text)
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(Y.shape)
print(X[0])
print(Y[0])

(120000,)
(120000,)
Wall Bears Claw Back Into Black Reuters Reuters Shortsellers 
2


In [3]:
f=open('data/ag_news_csv/test.csv','r')
lines=f.readlines()
YTest=[]
XTest=[]
for line in lines:
    line=line.split(',')
#     -1 to start index from 0
    YTest.append(int(line[0].strip("\""))-1)
    text=line[1].strip("\"")+" "+line[2].strip("\"")
    text=re.sub('[^ a-zA-Z]', '', text)
    text= pattern.sub('', text)
    result=""
    for word in text.split():
        if len(word)>=4:
            result+=lemma.lemmatize(word)+" "
    text=result
    XTest.append(text)
XTest=np.array(XTest)
YTest=np.array(YTest)
print(XTest.shape)
print(YTest.shape)
print(XTest[0])
print(YTest[0])

(7600,)
(7600,)
Fears pension talk Unions representing worker Turner Newall disappointed talk stricken parent firm Federal Mogul 
2


In [4]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = [w.lower() for w in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
#         if len(sentence_words)>=78:
#             print(len(sentence_words))
#             print(i)
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            if w in word_to_index.keys():
                X_indices[i, j] = word_to_index[w]
                j += 1
                if j>=25:
                    break;
            # Increment j to j + 1
            
            
    ### END CODE HERE ###
    
    return X_indices

In [5]:
def read_glove_vecs():
    with open('data/glove.6B.50d.txt','r',encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [6]:
softmax([1,2,3])

array([0.09003057, 0.24472847, 0.66524096])

In [7]:
maxLen=25
# for i in X:
    
#     if len(i)>maxLen:
#         maxLen=len(i)
# # maxLen = len(max(X, key=len).split())
print(maxLen)

25


In [8]:
Y_oh_train = convert_to_one_hot(Y, C = 4)
Y_oh_test = convert_to_one_hot(YTest, C = 4)
# Y147=np.array(Y147)
# Y173=np.array(Y173)
# Y161=np.array(Y161)
# Y162=np.array(Y162)
# Y_oh_147=convert_to_one_hot(Y147, C = 5)
# Y_oh_173=convert_to_one_hot(Y173, C = 5)
# Y_oh_161=convert_to_one_hot(Y161, C = 5)
# Y_oh_162=convert_to_one_hot(Y162, C = 5)
print(Y_oh_train)

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


In [9]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs()

In [10]:
word = "cucumber"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])
print(word_to_vec_map["cucumber"])

the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos
[ 0.68224  -0.31608  -0.95201   0.47108   0.56571   0.13151   0.22457
  0.094995 -1.3237   -0.51545  -0.39337   0.88488   0.93826   0.22931
  0.088624 -0.53908   0.23396   0.73245  -0.019123 -0.26552  -0.40433
 -1.5832    1.1316    0.4419   -0.48218   0.4828    0.14938   1.1245
  1.0159   -0.50213   0.83831  -0.31303   0.083242  1.7161    0.15024
  1.0324   -1.5005    0.62348   0.54508  -0.88484   0.53279  -0.085119
  0.02141  -0.56629   1.1463    0.6464    0.78318  -0.067662  0.22884
 -0.042453]


In [11]:
X1 = np.array(["sonu lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['sonu lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[337638. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


In [12]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [14]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -0.3403


In [15]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(4)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [16]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 25)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 25, 50)            20000050  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 256)           183296    
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
__________

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(X[56167])

Flow Interstellar Helium Solar System European Space Agency Consensus condition cloud interstellar surrounding several insitu observation method Through coordinated observation instrument several NASA spacecraft collaborative analysis effort hosted International Space Science Institute ISSI international team scientist compiled first time consistent physical parameter helium local interstellar cloud surround solar system Careful analysis data three complementary observation method produced reliable physical parameter local cloud used model interaction surrounding interstellar establish dimension solar system 


In [18]:
X_train_indices = sentences_to_indices(X, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y, C = 4)

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 512, shuffle=True)

Epoch 1/10
   512/120000 [..............................] - ETA: 78:27:21 - loss: 1.3913 - acc: 0.2617

In [ ]:
X_test_indices = sentences_to_indices(XTest, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(YTest, C = 4)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

In [ ]:
from keras.models import load_model

model.save('my_model10epochApproach3.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy")
print(sess.run(accuracy, feed_dict={x:X_test_avg,y_:Y_oh_test}))


In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'model/model.ckpt')


In [ ]:
sess.close()

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, 'model/model.ckpt')

In [ ]:
print(sess.run(accuracy, feed_dict={x:X_test_avg,y_:Y_oh_test}))

In [ ]:
print(sess.run(accuracy, feed_dict={x:X_train_avg,y_:Y_oh_train}))
# print(sess.run(accuracy, feed_dict={x:X147_avg,y_:Y_oh_147}))

# print(sess.run(accuracy, feed_dict={x:X173_avg,y_:Y_oh_173}))

# print(sess.run(accuracy, feed_dict={x:X162_avg,y_:Y_oh_162}))

# print(sess.run(accuracy, feed_dict={x:X161_avg,y_:Y_oh_161}))

In [ ]:

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy")
print(sess.run(accuracy, feed_dict={x:X_test_avg,y_:Y_oh_test}))

In [ ]:
predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage, feed_dict={x:X162_avg,y_:Y_oh_162}))
print(sess.run(correct_prediction, feed_dict={x:X162_avg,y_:Y_oh_162}))
print(sess.run(predictedIndex, feed_dict={x:X162_avg,y_:Y_oh_162}))
f=open('predictions162.txt','w')
f.write(str(sess.run(predictionPercentage, feed_dict={x:X162_avg,y_:Y_oh_162})))
f.write(str(sess.run(correct_prediction, feed_dict={x:X162_avg,y_:Y_oh_162})))
f.write(str(sess.run(predictedIndex, feed_dict={x:X162_avg,y_:Y_oh_162})))
f.close()

In [ ]:
predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage, feed_dict={x:X147_avg,y_:Y_oh_147}))
print(sess.run(correct_prediction, feed_dict={x:X147_avg,y_:Y_oh_147}))
print(sess.run(predictedIndex, feed_dict={x:X147_avg,y_:Y_oh_147}))
f=open('predictions147.txt','w')
f.write(str(sess.run(predictionPercentage, feed_dict={x:X147_avg,y_:Y_oh_147})))
f.write(str(sess.run(correct_prediction, feed_dict={x:X147_avg,y_:Y_oh_147})))
f.write(str(sess.run(predictedIndex, feed_dict={x:X147_avg,y_:Y_oh_147})))
f.close()

In [ ]:
predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage, feed_dict={x:X173_avg,y_:Y_oh_173}))
print(sess.run(correct_prediction, feed_dict={x:X173_avg,y_:Y_oh_173}))
print(sess.run(predictedIndex, feed_dict={x:X147_avg,y_:Y_oh_173}))
f=open('predictions173.txt','w')
f.write(str(sess.run(predictionPercentage, feed_dict={x:X173_avg,y_:Y_oh_173})))
f.write(str(sess.run(correct_prediction, feed_dict={x:X173_avg,y_:Y_oh_173})))
f.write(str(sess.run(predictedIndex, feed_dict={x:X147_avg,y_:Y_oh_173})))
f.close()

In [ ]:
predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage, feed_dict={x:X161_avg,y_:Y_oh_161}))
print(sess.run(correct_prediction, feed_dict={x:X161_avg,y_:Y_oh_161}))
print(sess.run(predictedIndex, feed_dict={x:X161_avg,y_:Y_oh_161}))
f=open('predictions161.txt','w')
f.write(str(sess.run(predictionPercentage, feed_dict={x:X161_avg,y_:Y_oh_161})))
f.write(str(sess.run(correct_prediction, feed_dict={x:X161_avg,y_:Y_oh_161})))
f.write(str(sess.run(predictedIndex, feed_dict={x:X161_avg,y_:Y_oh_161})))
f.close()

In [ ]:

newDataX=[]
newDataY=[]
path = './sports.txt'
files=glob.glob(path)
for file in files:
    f=open(file,'r',encoding="utf8")
    content=f.read().lower()
    content=re.sub('[^ a-zA-Z]', '', content)
    content= pattern.sub('', content)
    result=""
    for word in content.split():
        result+=lemma.lemmatize(word)+" "
    content=result
    newDataX.append(content)
    newDataY.append(4)
newDataXavg=[]
for i in range(0,np.shape(newDataX)[0]):
    newDataXavg.append(sentence_to_avg(newDataX[i], word_to_vec_map))
newDataY=np.array(newDataY)
newDataY_oh=convert_to_one_hot(newDataY, C = 5)

predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage,feed_dict={x:newDataXavg,y_:newDataY_oh}))
print(sess.run(predictedIndex,feed_dict={x:newDataXavg,y_:newDataY_oh}))
#print(sess.run(correct_prediction,feed_dict={x:newDataXavg,y_:newDataY_oh}))
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy")
print(sess.run(accuracy, feed_dict={x:newDataXavg,y_:newDataY_oh}))

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
# nltk.download('stopwords')
# nltk.download('punkt')
example_sent = "This is a sample sentence, showing off the stop words filtration."
 
stop_words = set(stopwords.words('english'))

pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
text = pattern.sub('', example_sent)
print(text)
nltk.download('wordnet')
result=""
for word in text.split():
    result+=lemma.lemmatize(word)+" "
    
print(result)

In [ ]:
saver = tf.train.Saver()
saver.save(sess, 'model/model.ckpt')
sess.close()

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, 'model/model.ckpt')

In [ ]:
newDataX=[]
newDataY=[]
path = './177/*.txt'
files=glob.glob(path)
for file in files:
    f=open(file,'r')
    content=f.read().lower()
    content=re.sub('[^ a-zA-Z]', '', content)
    content= pattern.sub('', content)
    result=""
    for word in content.split():
        result+=lemma.lemmatize(word)+" "
    content=result
    newDataX.append(content)
    newDataY.append(3)
newDataXavg=[]
for i in range(0,np.shape(newDataX)[0]):
    newDataXavg.append(sentence_to_avg(newDataX[i], word_to_vec_map))
newDataY=np.array(newDataY)
newDataY_oh=convert_to_one_hot(newDataY, C = 5)

predictionPercentage = tf.reduce_max(y, 1)*100
predictedIndex=tf.argmax(y,1)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(sess.run(predictionPercentage,feed_dict={x:newDataXavg,y_:newDataY_oh}))
print(sess.run(predictedIndex,feed_dict={x:newDataXavg,y_:newDataY_oh}))
#print(sess.run(correct_prediction,feed_dict={x:newDataXavg,y_:newDataY_oh}))
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("accuracy")
print(sess.run(accuracy, feed_dict={x:newDataXavg,y_:newDataY_oh}))